# IDENTIFICACIÓN DE LAS CARACTERÍSTICAS

En este archivo se crean las características de este estudio en los archivos train y test que serán utilizados posteriormente en los modelos predictivos.

In [13]:
import pandas as pd

## TRAIN

Carga de los datos para nuestro archivo 'train.xlsx'.

In [14]:
#Para introducir los datos de las diferentes semanas:
#Semana 1: df_train_xlsx = './Datos_TFG/datos_correctos/semanas/SEMANA1_81_89_96.xlsx'
#Semana 2: df_train_xlsx = './Datos_TFG/datos_correctos/semanas/SEMANA2_81_89_96.xlsx'
#Semana 3: df_train_xlsx = './Datos_TFG/datos_correctos/semanas/SEMANA3_81_89_96.xlsx'
#Semana 4: df_train_xlsx = './Datos_TFG/datos_correctos/semanas/SEMANA4_81_89_96.xlsx'
#Semana 5: df_train_xlsx = './Datos_TFG/datos_correctos/semanas/SEMANA5_81_89_96.xlsx'

df_train_xlsx = './Datos_TFG/datos_correctos/semanas/SEMANA4_81_89_96.xlsx'
df_train = pd.read_excel(df_train_xlsx)
df_train.head()

,login_est,fecha,hora,moodle_component,moodle_resource_nombre,moodle_action,moodle_target,process
0,59,2021-08-30,16:59:30,core,NaN,viewed,course,web
1,59,2021-08-30,17:05:30,core,NaN,viewed,course,web
2,78,2021-08-30,10:27:35,core,NaN,viewed,course,web
3,78,2021-08-30,10:27:40,gradereport_user,NaN,viewed,grade_report,web
4,78,2021-08-30,20:01:09,core,NaN,viewed,course,web


Conversión de la columna 'fecha' y 'hora' a datetime

In [15]:
df_train['fecha'] = pd.to_datetime(df_train['fecha'], format='%d/%m/%Y')
df_train['hora'] = pd.to_datetime(df_train['hora'], format='%H:%M:%S')
df_train.tail()


,login_est,fecha,hora,moodle_component,moodle_resource_nombre,moodle_action,moodle_target,process
23289,128,2021-10-03,1900-01-01 15:26:24,mod_resource,NaN,viewed,course_module,web
23290,128,2021-10-03,1900-01-01 15:26:46,mod_resource,NaN,viewed,course_module,web
23291,128,2021-10-03,1900-01-01 16:33:01,mod_assign,NaN,submitted,assessable,web
23292,128,2021-10-03,1900-01-01 16:33:01,assignsubmission_file,NaN,uploaded,assessable,web
23293,128,2021-10-03,1900-01-01 16:33:01,assignsubmission_file,NaN,created,submission,web


Función diasEntrega(df, semana)

Esta función permite calcular los días que pasan desde que un alumno visualiza unos ejercicios por primera vez hasta que sube la solución a la plataforma.

In [16]:
def diasEntrega(df, semana):
    filtro_semana_viewed = df[(df['moodle_resource_nombre'] == semana) & (df['moodle_action'] == 'viewed')]
    primer_viewed = filtro_semana_viewed.groupby('login_est')['fecha'].min()
    filtro_submitted = df[(df['moodle_resource_nombre'].isna()) & (df['moodle_action'] == 'submitted')]
    #Almacenamos el primer 'submitted' que sigue a 'viewed'.
    primer_submitted_despues_viewed = pd.Series(index=primer_viewed.index, dtype='datetime64[ns]')
    #Para cada estudiante, se selecciona el primer 'submitted' que ocurra después del 'viewed'.
    for estudiante, fecha_viewed in primer_viewed.items():
        submitted_post_viewed = filtro_submitted[(filtro_submitted['login_est'] == estudiante) & 
                                            (filtro_submitted['fecha'] > fecha_viewed)]['fecha']
        if not submitted_post_viewed.empty:
            primer_submitted_despues_viewed[estudiante] = submitted_post_viewed.min()
    nuevo_df = pd.DataFrame({
        'primer_viewed': primer_viewed,
        'primer_uploaded': primer_submitted_despues_viewed
    })
    #Calculamos la diferencia en días.
    nuevo_df['diferencia_dias'] = (nuevo_df['primer_uploaded'] - nuevo_df['primer_viewed']).dt.days
    return nuevo_df

Función conteoActividades(df, actividad)

Esta función permite contar el número de accesos que los alumnos han hecho de una actividad.

In [17]:
def conteoActividades(df, actividad):
    #Se filtra por la actividad que ve el estudiante y 'moodle_action' = viewed.
    filtro_actividad = df[(df['moodle_resource_nombre'] == actividad) & (df['moodle_action'] == 'viewed')]
    conteo_accesos = filtro_actividad.groupby('login_est').size().rename(f'accesos_{actividad}')
    return conteo_accesos

Función calcular_promedio_wooclap(df, columnas)

Esta función permite calular el promedio de los wooclaps de los alumnos.

In [18]:
def calcular_promedio_wooclap(df, columnas):
    # Se calcula la media de los Wooclap para cada estudiante.
    df['wooclap_average'] = df[columnas].mean(axis=1)
    return df[['wooclap_average', 'nota_final_aprobado']]

Variables que se utilizan en los dos conjuntos de entrenamiento y prueba como atributos de las funciones detalladas anteriormente.

In [19]:
#Estas variables para los datos de las diferentes semanas son:
#Semana 1: semanas = ['Week 1: Algorithms'], acciones_excluidas = ['added', 'updated'], wooclaps = []
#Semana 2: semanas = ['Week 1: Algorithms','Week 2: Flow diagrams'], acciones_excluidas = ['added', 'updated'], wooclaps = ['wooclap_week2']
#Semana 3: semanas = ['Week 1: Algorithms','Week 2: Flow diagrams','Week 3: Variables and arithmetic operators'], acciones_excluidas = ['added', 'updated'], wooclaps = ['wooclap_week2','wooclap_week3']
#Semana 4: semanas = ['Week 1: Algorithms','Week 2: Flow diagrams','Week 3: Variables and arithmetic operators','Week 4: operators, casting and decision-making'], acciones_excluidas = ['added', 'updated'], wooclaps = ['wooclap_week2','wooclap_week3','wooclap_week4']
#Semana 5: semanas = ['Week 1: Algorithms','Week 2: Flow diagrams','Week 3: Variables and arithmetic operators','Week 4: operators, casting and decision-making','Week 5: loops'], acciones_excluidas = ['added', 'updated'], wooclaps = ['wooclap_week2','wooclap_week3','wooclap_week4','wooclap_week5']

#La variable semanas contiene una lista con los ejercicios de las semanas 1, 2, 3, 4 y 5. Los posibles valores que se pueden añadir son: 'Week 1: Algorithms', 'Week 2: Flow diagrams',
#'Week 3: Variables and arithmetic operators', 'Week 4: operators, casting and decision-making', 'Week 5: loops'.
semanas = [
    'Week 1: Algorithms',
    'Week 2: Flow diagrams',
    'Week 3: Variables and arithmetic operators',
    'Week 4: operators, casting and decision-making'
]

#Acciones excluidas.
acciones_excluidas = [
    'added', 
    'updated'
]

#La variable wooclaps contiene una lista con las notas de los wooclap de cada semana. Los posibles valores que se pueden añadir son: 'wooclap_week2', 'wooclap_week3', 'wooclap_week4', 'wooclap_week5'.
wooclaps = [
    'wooclap_week2',
    'wooclap_week3',
    'wooclap_week4'
]

Se construyen las características para el archivo TRAIN

In [20]:
#Días entre que se visualiza por priemra vez los ejercicios hasta que se manda el archivo mediante la función diasEntrega().
diferencias_dias = {}
for i, semana in enumerate(semanas, 1):
    resultados = diasEntrega(df_train, semana)
    diferencias_dias[f'diferencia_dias_week{i}'] = resultados['diferencia_dias']

diferencias_dias_train = pd.DataFrame(diferencias_dias)

#Número de veces que se visualiza una actividad a través de la función conteoActividades().
conteos = {}
for i, semana in enumerate(semanas, 1):
    resultados = conteoActividades(df_train, semana)
    conteos[f'accesos_week{i}'] = resultados
conteos_train = pd.DataFrame(conteos).fillna(0)

#Conteo de cada una de las acciones que realiza cada alumno cada vez que entra a la plataforma sin tener en cuenta algunas acciones.
df_train_add = df_train[~df_train['moodle_action'].isin(acciones_excluidas)]
frecuencia_acciones = df_train_add.pivot_table(index='login_est', columns='moodle_action', aggfunc='size', fill_value=0)

#Número de interacciones de cada alumno por día de la semana.
df_train['dia_semana'] = df_train['fecha'].dt.day_name()
interacciones_dia = pd.get_dummies(df_train['dia_semana']).groupby(df_train['login_est']).sum()

#Se añaden las características 'nota_final_aprobado' y 'wooclaps' al dataframe.
datos_xlsx_train = './Datos_TFG/datos_correctos/notas_81_89_96.xlsx'
data_notas = pd.read_excel(datos_xlsx_train)
data_notas.set_index('Identificador', inplace=True)

#Nota final de los alumnos y media de los wooclaps de la semana.
nota_final_aprobado = calcular_promedio_wooclap(data_notas, wooclaps)

#Se unen todas las características.
caracteristicas_train = pd.DataFrame(frecuencia_acciones).join([interacciones_dia, diferencias_dias_train, conteos_train, nota_final_aprobado])
caracteristicas_train.fillna(-1, inplace=True)
pd.set_option('display.max_rows', 10)

#Finalmente, se exportan los datos en un archivo excel para su posterior uso en los diferentes modelos.
trainxlsx = 'train.xlsx'
caracteristicas_train.to_excel(trainxlsx, index=True)


### TEST

Carga de los datos para nuestro archivo 'test.xlsx'

In [21]:
#Para introducir los datos de las diferentes semanas:
#Semana 1: df_test_xlsx = './Datos_TFG/datos_correctos/semanas/SEMANA1_84.xlsx'
#Semana 2: df_test_xlsx = './Datos_TFG/datos_correctos/semanas/SEMANA2_84.xlsx'
#Semana 3: df_test_xlsx = './Datos_TFG/datos_correctos/semanas/SEMANA3_84.xlsx'
#Semana 4: df_test_xlsx = './Datos_TFG/datos_correctos/semanas/SEMANA4_84.xlsx'
#Semana 5: df_test_xlsx = './Datos_TFG/datos_correctos/semanas/SEMANA5_84.xlsx'

df_test_xlsx = './Datos_TFG/datos_correctos/semanas/SEMANA4_84.xlsx'
df_test = pd.read_excel(df_test_xlsx)
df_test.head()

,login_est,fecha,hora,moodle_component,moodle_resource_nombre,moodle_action,moodle_target,process
0,137,2021-08-30,11:05:24,core,NaN,viewed,course,web
1,137,2021-08-30,11:15:31,core,NaN,viewed,course,web
2,137,2021-08-30,11:17:12,core,NaN,viewed,course,web
3,137,2021-08-30,11:05:32,mod_url,NaN,viewed,course_module,web
4,144,2021-08-30,07:35:19,core,NaN,viewed,course,web


Se convierte la columna 'fecha' y 'hora' a datetime

In [22]:
df_test['fecha'] = pd.to_datetime(df_test['fecha'], format='%Y-%m-%d')
df_test['hora'] = pd.to_datetime(df_test['hora'], format='%H:%M:%S')
df_test.tail()

,login_est,fecha,hora,moodle_component,moodle_resource_nombre,moodle_action,moodle_target,process
4763,139,2021-10-03,1900-01-01 18:23:13,assignsubmission_file,NaN,created,submission,web
4764,143,2021-10-03,1900-01-01 23:02:47,assignsubmission_file,NaN,created,submission,web
4765,130,2021-10-03,1900-01-01 21:22:52,assignsubmission_file,NaN,created,submission,web
4766,150,2021-10-03,1900-01-01 20:11:55,assignsubmission_file,NaN,created,submission,web
4767,148,2021-10-03,1900-01-01 20:12:04,assignsubmission_file,NaN,created,submission,web


Se construyen las características para el archivo TEST

In [23]:
#Días entre que se visualiza por priemra vez los ejercicios hasta que se manda el archivo mediante la función diasEntrega().
diferencias_dias = {}
for i, semana in enumerate(semanas, 1):
    resultados = diasEntrega(df_test, semana)
    diferencias_dias[f'diferencia_dias_week{i}'] = resultados['diferencia_dias']

diferencias_dias_test = pd.DataFrame(diferencias_dias)

#Número de veces que se visualiza una actividad a través de la función conteoActividades().
conteos = {}
for i, semana in enumerate(semanas, 1):
    resultados = conteoActividades(df_test, semana)
    conteos[f'accesos_week{i}'] = resultados
conteos_test = pd.DataFrame(conteos).fillna(0)

#Conteo de cada una de las acciones que realiza cada alumno cada vez que entra a la plataforma sin tener en cuenta algunas acciones.
df_test_add = df_test[~df_test['moodle_action'].isin(acciones_excluidas)]
frecuencia_acciones = df_test_add.pivot_table(index='login_est', columns='moodle_action', aggfunc='size', fill_value=0)

#Número de interacciones de cada alumno por día de la semana
df_test['dia_semana'] = df_test['fecha'].dt.day_name()
interacciones_dia = pd.get_dummies(df_test['dia_semana']).groupby(df_test['login_est']).sum()

#Se añaden las características 'nota_final_aprobado' y 'wooclaps' al dataframe.
datos_xlsx_test = './Datos_TFG/datos_correctos/notas_84.xlsx'
data_notas = pd.read_excel(datos_xlsx_test)
data_notas.set_index('Identificador', inplace=True)

#Nota final de los alumnos y media de los wooclaps de la semana.
nota_final_aprobado = calcular_promedio_wooclap(data_notas, wooclaps)

#Se unen todas las características.
caracteristicas_test = pd.DataFrame(frecuencia_acciones).join([interacciones_dia, diferencias_dias_test, conteos_test, nota_final_aprobado])
caracteristicas_test.fillna(-1, inplace=True)
pd.set_option('display.max_rows', 10)

#Finalmente, se expoprtan los datos en un archivo excel para su posterior uso en los diferentes modelos.
testxlsx = 'test.xlsx'
caracteristicas_test.to_excel(testxlsx, index=True)